In [ ]:
!pip install -r requirements.txt

In [ ]:
%run ./data.ipynb

In [ ]:
"""Define models."""

import keras
import numpy as np

seed = 10
np.random.seed(seed)


def simple_model(input_dim: int, output_dim: int):
    model = keras.models.Sequential(
        [
            keras.layers.Dense(15, input_shape=(input_dim,), activation='relu'),
            keras.layers.Dense(15, activation='relu'),
            keras.layers.Dense(output_dim, activation='softmax'),
        ]
    )
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam',
        metrics=['accuracy'],
    )
    
    return model

In [ ]:
model = simple_model(
    input_dim=x_ar.shape[1],
    output_dim=y_ar.shape[1],
)

model.summary()

checkpoint = keras.callbacks.ModelCheckpoint(
    filepath="../models/simple-{epoch:02d}-{val_loss:.2f}.hdf5",
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    mode="auto",
    save_freq="epoch",
)

history = model.fit(
    x_ar,
    y_ar,
    batch_size=100,
    epochs=2,
    shuffle=True,
    validation_split=0.2,
    callbacks=[checkpoint],
)

In [ ]:
"""Plot accuracies."""

import matplotlib.pyplot as plt

def plot_accuracies(history):
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    
    plt.title('model accuracy')
    
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    
    plt.legend(['train', 'test'])
    
    plt.savefig('accuracy')    
    
plot_accuracies(history.history)

In [ ]:
"""Define cross validation schema."""

from sklearn.model_selection import KFold

# KFold Cross Validation
kfold = KFold(
    n_splits=5, 
    shuffle=True,
    random_state=seed,
)

In [ ]:
"""Define training schema (estimator)."""

from keras.wrappers.scikit_learn import KerasClassifier

estimator = KerasClassifier(
    build_fn=lambda: baseline_model(
        input_dim=x_ar.shape[1],
        output_dim=y_ar.shape[1],
    ), 
    epochs=1,
    batch_size=1000,
    verbose=0,
)

In [ ]:
"""Train estimator on cross-validation schema and collect results."""

import sklearn.model_selection 

results = sklearn.model_selection.cross_val_score(
    estimator, 
    x_ar, 
    y_ar, 
    cv=kfold,
)

print(results)

print(
    "Result: %.2f%% (%.2f%%)" % (
        results.mean()*100, 
        results.std()*100,
    )
)